In [1]:
from mailchimp3 import MailChimp
import pandas as pd
import os, json

date_format = '%Y-%m-%d %H:%M:%S'

In [2]:
# conexion con mailchimp API
client = MailChimp(mc_api='064e35a5ec6d76812f4877ac3c910561-us17', mc_user='vitamina.rcd@gmail.com')

In [ ]:
# obtener todas las campañas
response = client.campaigns.all(get_all=True)
campaings = response['campaigns']

In [ ]:
# preparar dict para recibir campañas
all_campaings = []

for i, campaing in enumerate(campaings):
    
    campaing_web_id = campaing['web_id']
    campaing_status = campaing['status']
    campaing_create_time = pd.to_datetime(campaing['create_time'])
    campaing_create_time = campaing_create_time.strftime(date_format)
    list_id = campaing['recipients']['list_id']
    list_is_active = campaing['recipients']['list_is_active']
    list_name = campaing['recipients']['list_name']
    list_total_contacts = campaing['recipients']['recipient_count']
    try:
        email_subject = campaing['settings']['subject_line']
    except:
        email_subject = ''
    campaing_title = campaing['settings']['title']
    try:
        campaing_from_name = campaing['settings']['from_name']
    except:
        campaing_from_name = ''
    try:
        campaing_reply_to = campaing['settings']['reply_to']
    except:
        campaing_reply_to = ''
    campaing_template_id = campaing['settings']['template_id']
    try:
        report_opens_value = campaing['report_summary']['opens']
        report_unique_opens_value = campaing['report_summary']['unique_opens']
        report_open_rate = campaing['report_summary']['open_rate']
        report_click_rate = campaing['report_summary']['click_rate']
    except:
        report_opens_value = 0
        report_unique_opens_value = 0
        report_open_rate = 0
        report_click_rate = 0
        
    all_campaings.append(
        {
            'campaing_web_id' : campaing_web_id,
            'campaing_status' : campaing_status,
            'campaing_create_date' : campaing_create_time,
            'list_id' : list_id,
            'list_is_active' : list_is_active,
            'list_name' : list_name,
            'list_total_contacts' : list_total_contacts,
            'email_subject' : email_subject,
            'campaing_title' : campaing_title,
            'campaing_from_name' : campaing_from_name,
            'campaing_reply_to' : campaing_reply_to,
            'campaing_template_id' : campaing_template_id,
            'report_opens_value' : report_opens_value,
            'report_unique_opens_value': report_unique_opens_value,
            'report_open_rate' : report_open_rate,
            'report_click_rate' : report_click_rate
         }
    )
all_campaings = pd.DataFrame(all_campaings)

In [ ]:
adaptacion_campaings = all_campaings[all_campaings['campaing_title'].str.contains('Adaptación')]
adaptacion_campaings = adaptacion_campaings.sort_values('campaing_web_id', ascending=False)
adaptacion_campaings.head()

In [ ]:
# exportar campañas de adaptacion a excel para revision
adaptacion_campaings.to_excel('campañas_adaptacion_encontradas_mailchimp.xlsx', index=False)

In [15]:
#Funcion para crear audiencia segun campos solicitados

def audience_creation_function(audience_creation_dictionary):
        
    #prepare de variable to receive the audience creation
    audience_creation = ''

    #defining a variable with the dictionary that we need to create
    audience_creation_dictionary = audience_creation_dictionary
    print(audience_creation_dictionary)

    audience_list = {
        "name": audience_creation_dictionary['audience_name'],
        "contact":
        {
            "company": audience_creation_dictionary['company'],
            "address1": audience_creation_dictionary['address1'],
            "address2": audience_creation_dictionary['address2'],
            "city": audience_creation_dictionary['city'],
            "state": audience_creation_dictionary['state'],
            "zip": audience_creation_dictionary['zip_code'],
            "country": audience_creation_dictionary['country']
        },
        "permission_reminder": '.',
        "campaign_defaults":
        {
            "from_name": audience_creation_dictionary['from_name'],
            "from_email": audience_creation_dictionary['from_email'],
            "subject": "",
            "language": audience_creation_dictionary['language']
        },
        "email_type_option": False
    }

    try:
        audience_creation = client.lists.create(data = audience_list)
    except Exception as error:
        print(error)
        
    return audience_creation


def add_members_to_audience_function(audience_id, mail_list, client=client):
        
    audience_id = audience_id
    # debe ser un dataframe
    email_list = mail_list

    #merging values before to add contacts

    # merge campos de la data
    client.lists.merge_fields.create(list_id=audience_id, data={
        'tag': 'TAG',
        'name': 'TAG',
        'type': 'text',
        'required': False,
        'default_value': '',
        'public': True
    })


    client.lists.merge_fields.create(list_id=audience_id, data={
        'tag': 'TIPO',
        'name': 'TIPO',
        'type': 'text',
        'required': False,
        'default_value': '',
        'public': True
    })

    if len(email_list)!=0:
        for i_, email_iteration in email_list.iterrows():
            try:
                data = {
                    "email_address" : email_iteration['Email Address'],
                    "status": "subscribed",
                    "merge_fields" : {
                        "TIPO" : email_iteration['TIPO'],
                        "TAG" : email_iteration['TAG']
                    }
                        
                }
                client.lists.members.create(list_id=audience_id, data=data)
                print('{} has been successfully added to the {} audience'.format(email_iteration, audience_id))

            except Exception as error:
                print(error)
    else: 
        print('Email list is empty')


def campaign_creation_function(campaign_name, audience_id, subject, from_name, reply_to, template_id=10000034, client=client):
        
    campaign_name = campaign_name
    audience_id = audience_id
    from_name = from_name
    reply_to = reply_to

    data = {
        "content_type": 'template',
        "recipients" :
        {
            "list_id": audience_id
        },
        "settings":
        {   
            "title": campaign_name,
            "subject_line": subject,
            "from_name": from_name,
            "reply_to": reply_to,
            'template_id': template_id,
        },
        "type": "regular"
    }

    new_campaign = client.campaigns.create(data=data)
    
    return new_campaign


def send_mail(campaign_id, client=client):      
    try:
        client.campaigns.actions.send(campaign_id = campaign_id)
    except Exception as error:
        print(error)

In [4]:
# diccionario para crear audiencia
audience_creation_dictionary = {
    "audience_name" : "Vitamina | test numero 2 creacion lista api python | 20230126",
    "company" : "Vitamina | Salas Cuna y Jardines Infantiles",
    "address1" : "Oficina 1801",
    "address2" : "Avenida Apoquindo 4501",
    "state" : "Region Metropolitana",
    "city" :  "Las Condes",
    "zip_code" : "7550000",
    "country" : "CL", # EX: LK
    "from_name" : "Vitamina",
    "from_email" : "experiencia@vitamina.cl",
    "language" : "es"
} 

In [5]:
# creacion de audiencia
audience_creation = audience_creation_function(audience_creation_dictionary)

{'audience_name': 'Vitamina | test numero 2 creacion lista api python | 20230126', 'company': 'Vitamina | Salas Cuna y Jardines Infantiles', 'address1': 'Oficina 1801', 'address2': 'Avenida Apoquindo 4501', 'state': 'Region Metropolitana', 'city': 'Las Condes', 'zip_code': '7550000', 'country': 'CL', 'from_name': 'Vitamina', 'from_email': 'experiencia@vitamina.cl', 'language': 'es'}


In [6]:
# getting contacts to try
contacts = pd.read_csv('plantilla_prueba.csv')
contacts.head()

,Email Address,TIPO,TAG
0,jaime.arroyo@vitamina.com,ENADAP,183567942
1,ana.coloma@vitamina.com,ENADAP,967821231
2,camila.saa@vitamina.com,ENADAP,888888888
3,zoris.velasquez@vitamina.com,ENADAP,999999999
4,javiera.carter@vitamina.com,ENADAP,11111111111


In [7]:
# crear contactos a la audiencia
result = add_members_to_audience_function(audience_id=audience_creation['id'],mail_list=contacts)

Email Address    jaime.arroyo@vitamina.com
TIPO                                ENADAP
TAG                              183567942
Name: 0, dtype: object has been successfully added to the 50f6a293aa audience
Email Address    ana.coloma@vitamina.com
TIPO                              ENADAP
TAG                            967821231
Name: 1, dtype: object has been successfully added to the 50f6a293aa audience
Email Address    camila.saa@vitamina.com
TIPO                              ENADAP
TAG                            888888888
Name: 2, dtype: object has been successfully added to the 50f6a293aa audience
Email Address    zoris.velasquez@vitamina.com
TIPO                                   ENADAP
TAG                                 999999999
Name: 3, dtype: object has been successfully added to the 50f6a293aa audience
Email Address    javiera.carter@vitamina.com
TIPO                                  ENADAP
TAG                              11111111111
Name: 4, dtype: object has been successf

In [18]:
# Creacion de campaña
campaign_name = 'Prueba Vitamina numero 2 automatizacion envio mail via Python con Mailchip'
from_name = 'Vitamina'
reply_to = 'experiencia@vitamina.cl'
subject = '¿Cuéntanos cómo fueron tus primeros días en Vitamina? 🧡'
template_id = 10000034
campaign = campaign_creation_function(campaign_name=campaign_name,
                                      audience_id=audience_creation['id'],
                                      subject=subject,
                                      from_name=from_name,
                                      reply_to=reply_to,
                                      template_id=template_id
                                      client=client)

In [19]:
# Envio de campaña
send_mail(campaign_id=campaign['id'])